In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import tabula
%matplotlib inline

* participants in Wave IV are 24 - 32 years old
* Census data in thousands
* Wave 3: H2ED7 - 10, H2ED15 - 20, H2WP9 - 16, H2PF4-5, H2PF9-10, H2PR3-4

Census

In [2]:
census_all = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\Census Educational Attainment - 2008.xls", 'all')
census_all.head()

,All Races,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade /2,High school graduate,Some college no degree,"Associate's degree, occupational","Associate's degree, academic",Bachelor's degree,Master's degree,Professional degree,Doctoral degree
0,.18 years and over,224703,824,1878,3802,4927,4061,5415,11103,69480,44168,9016,9573,40070,14893,3009,2485
1,..18 to 24 years,28398,47,56,229,274,464,869,3731,8296,10356,608,799,2511,128,17,14
2,.25 years and over,196305,776,1822,3573,4653,3598,4546,7372,61183,33812,8408,8774,37559,14765,2991,2472
3,..25 to 29 years,21057,30,98,299,276,408,505,951,6017,4109,834,1054,5012,1227,181,57
4,..30 to 34 years,19089,40,114,345,269,395,330,708,5280,3287,873,956,4409,1565,309,210


In [3]:
censusall_sub = census_all.iloc[3:5]
censusall_sub

,All Races,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade /2,High school graduate,Some college no degree,"Associate's degree, occupational","Associate's degree, academic",Bachelor's degree,Master's degree,Professional degree,Doctoral degree
3,..25 to 29 years,21057,30,98,299,276,408,505,951,6017,4109,834,1054,5012,1227,181,57
4,..30 to 34 years,19089,40,114,345,269,395,330,708,5280,3287,873,956,4409,1565,309,210


In [4]:
censusall_sub.loc['Total'] = censusall_sub.iloc[:, :].sum()

C:\Users\mnorm\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [5]:
censusall_sub.reset_index(drop = False)

,index,All Races,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade /2,High school graduate,Some college no degree,"Associate's degree, occupational","Associate's degree, academic",Bachelor's degree,Master's degree,Professional degree,Doctoral degree
0,3,..25 to 29 years,21057,30,98,299,276,408,505,951,6017,4109,834,1054,5012,1227,181,57
1,4,..30 to 34 years,19089,40,114,345,269,395,330,708,5280,3287,873,956,4409,1565,309,210
2,Total,..25 to 29 years..30 to 34 years,40146,70,212,644,545,803,835,1659,11297,7396,1707,2010,9421,2792,490,267


In [6]:
censusall_sub.to_csv('census_school.csv')

In [7]:
census_pct = censusall_sub.iloc[1:, 1:].div(censusall_sub['Total'], axis = 0)

In [8]:
for col in censusall_sub.columns[2:] :   
    census_pct[col] = (censusall_sub[col]/censusall_sub['Total'])

In [9]:
census_pct = round(census_pct, 4)
census_pct

,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade /2,High school graduate,Some college no degree,"Associate's degree, occupational","Associate's degree, academic",Bachelor's degree,Master's degree,Professional degree,Doctoral degree
3,NaN,0.0014,0.0047,0.0142,0.0131,0.0194,0.0240,0.0452,0.2857,0.1951,0.0396,0.0501,0.2380,0.0583,0.0086,0.0027
4,1.0,0.0021,0.0060,0.0181,0.0141,0.0207,0.0173,0.0371,0.2766,0.1722,0.0457,0.0501,0.2310,0.0820,0.0162,0.0110
Total,1.0,0.0017,0.0053,0.0160,0.0136,0.0200,0.0208,0.0413,0.2814,0.1842,0.0425,0.0501,0.2347,0.0695,0.0122,0.0067


Merge Wave AIDs

In [10]:
wave1 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID'])
wave2 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0005\21600-0005-Data.tsv", sep = '\t', usecols = ['AID'])
wave3educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID'])
wave3grad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0017\21600-0017-Data.tsv", sep = '\t', usecols = ['AID'])
wave4 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID'])

In [11]:
print('Count of Responses per Wave')
print('Wave 1: ', len(wave1))
print('Wave 3 education: ', len(wave3educ))
print('Wave 3 graduation: ', len(wave3grad))
print('Wave 4: ', len(wave4))

Count of Responses per Wave
Wave 1:  6504
Wave 3 education:  3947
Wave 3 graduation:  4521
Wave 4:  5114


In [12]:
waves = [wave1, wave3educ, wave3grad, wave4]
AID = reduce(lambda left, right: pd.merge(left, right, on = 'AID', how = 'inner'), waves)
len(AID)

3442

Highest Education Level

In [13]:
highest_educ = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'highest_educ_level')
highest_educ = highest_educ.rename(columns = highest_educ.iloc[0])
highest_educ = highest_educ.drop(highest_educ.index[0])
#highest_educ = highest_educ.drop(['Value'], axis = 1)
highest_educ

,Frequency,Percent,Value,Label
1,16,0.3%,1,8th grade or less
2,383,7.5%,2,some high school
3,835,16.3%,3,high school graduate
4,182,3.6%,4,some vocational/technical training (after high...
5,327,6.4%,5,completed vocational/technical training (after...
6,1702,33.3%,6,some college
7,1012,19.8%,7,completed college (bachelor's degree)
8,199,3.9%,8,some graduate school
9,256,5.0%,9,completed a master's degree
10,59,1.2%,10,some graduate training beyond a master's degree


In [14]:
highest_educ.to_csv('highest_educ.csv')

In [15]:
census_pct

,Total,None,1st - 4th grade,5th - 6th grade,7th - 8th grade,9th grade,10th grade,11th grade /2,High school graduate,Some college no degree,"Associate's degree, occupational","Associate's degree, academic",Bachelor's degree,Master's degree,Professional degree,Doctoral degree
3,NaN,0.0014,0.0047,0.0142,0.0131,0.0194,0.0240,0.0452,0.2857,0.1951,0.0396,0.0501,0.2380,0.0583,0.0086,0.0027
4,1.0,0.0021,0.0060,0.0181,0.0141,0.0207,0.0173,0.0371,0.2766,0.1722,0.0457,0.0501,0.2310,0.0820,0.0162,0.0110
Total,1.0,0.0017,0.0053,0.0160,0.0136,0.0200,0.0208,0.0413,0.2814,0.1842,0.0425,0.0501,0.2347,0.0695,0.0122,0.0067


In [16]:
census_pct.to_csv('census_pct.csv')

In [17]:
most_recent_degree = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'most_recent_degree')
most_recent_degree = most_recent_degree.rename(columns = most_recent_degree.iloc[0])
most_recent_degree = most_recent_degree.drop(most_recent_degree.index[0])
#most_recent_degree = most_recent_degree.drop(['Value'], axis =1)
most_recent_degree
#legitimate skip represents those who did not complete high school

,Frequency,Percent,Value,Description
1,1166,22.8%,1,have not received a degree or certificate
2,471,9.2%,2,"certificate or degree from a 1-, 2-, or 3-year..."
3,470,9.2%,3,associate's degree
4,1187,23.2%,4,bachelor's degree
5,34,0.7%,5,certificate from a 1- or 2-year post baccalaur...
6,280,5.5%,6,master's degree
7,14,0.3%,7,"PhD degree or equivalent (EDD, DrPH, etc.)"
8,75,1.5%,8,"professional doctorate (MD, JD, LLB, DDS, etc.)"
9,1416,27.7%,97,legitimate skip
10,1,0.0%,98,don't know


Did students who had greater academic success in hs (gpa above 3), attain a higher level of education?

In [18]:
hs_gpa = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID', 'EAOGPAC'])
hs_gpa = hs_gpa.rename(columns = {'AID': 'AID', 'EAOGPAC': 'cumulative_gpa'})
hs_gpa = pd.merge(AID, hs_gpa, on = 'AID', how = 'left')
#hs_gpa.isna().sum() #check for nulls to make sure the AID matched
bins = [-1, 1, 2, 3 ,4]
labels = ['0-1', '1-2', '2-3', '3-4']
hs_gpa['gpa_range'] = pd.cut(np.array(hs_gpa['cumulative_gpa']), bins = bins, labels = labels)
hs_gpa

,AID,cumulative_gpa,gpa_range
0,57103869,0.767,0-1
1,57111071,3.207,3-4
2,57117542,9995.000,NaN
3,57118381,3.156,3-4
4,57118943,2.912,2-3
...,...,...,...
3437,99718944,3.345,3-4
3438,99719378,3.118,3-4
3439,99719930,3.548,3-4
3440,99719939,2.871,2-3


In [19]:
hs_gpa.dtypes

AID                  int64
cumulative_gpa     float64
gpa_range         category
dtype: object

In [20]:
#remove students with 9995 value as cumulative gpa. those students didn't receive grades
#len(hs_gpa.loc[hs_gpa['cumulative_gpa'] == 9995]) =24 students
hs_gpa = hs_gpa.drop(hs_gpa[hs_gpa['cumulative_gpa'] == 9995].index)

In [21]:
educ_attainment = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4ED3A'])
educ_attainment = educ_attainment.rename(columns = {'H4ED3A':'Value'})
educ_attainment.dtypes

AID      int64
Value    int64
dtype: object

In [22]:
mrd_code = most_recent_degree[['Value', 'Description']]
mrd_code.dtypes

Value          object
Description    object
dtype: object

In [23]:
mrd_code['Value'] = mrd_code['Value'].astype('int')

C:\Users\mnorm\AppData\Local\Temp/ipykernel_34520/3151247090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrd_code['Value'] = mrd_code['Value'].astype('int')


In [24]:
educ_attainment = pd.merge(educ_attainment, mrd_code, on = 'Value', how = 'left')

In [25]:
gpa_educ = pd.merge(hs_gpa, educ_attainment, on = 'AID', how = 'left')
#gpa_educ.isna().sum()
bins = [-1, 1, 2, 3 ,4]
labels = ['0-1', '1-2', '2-3', '3-4']
gpa_educ['gpa_range'] = pd.cut(np.array(gpa_educ['cumulative_gpa']), bins = bins, labels = labels)
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].min())

,cumulative_gpa
Description,
"PhD degree or equivalent (EDD, DrPH, etc.)",2.673
associate's degree,0.769
bachelor's degree,1.000
certificate from a 1- or 2-year post baccalaureate academic program,1.613
"certificate or degree from a 1-, 2-, or 3-year vocational/technical\nprogram (after high school and not including associate's degree)",0.000
have not received a degree or certificate,0.233
legitimate skip,0.000
master's degree,1.898
"professional doctorate (MD, JD, LLB, DDS, etc.)",1.880


In [26]:
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].max())

,cumulative_gpa
Description,
"PhD degree or equivalent (EDD, DrPH, etc.)",4.000
associate's degree,4.000
bachelor's degree,4.000
certificate from a 1- or 2-year post baccalaureate academic program,4.000
"certificate or degree from a 1-, 2-, or 3-year vocational/technical\nprogram (after high school and not including associate's degree)",3.981
have not received a degree or certificate,4.000
legitimate skip,3.935
master's degree,4.000
"professional doctorate (MD, JD, LLB, DDS, etc.)",4.000


In [27]:
pd.DataFrame(gpa_educ.groupby('Description')['cumulative_gpa'].mean())

,cumulative_gpa
Description,
"PhD degree or equivalent (EDD, DrPH, etc.)",3.612364
associate's degree,2.719964
bachelor's degree,3.176535
certificate from a 1- or 2-year post baccalaureate academic program,3.060526
"certificate or degree from a 1-, 2-, or 3-year vocational/technical\nprogram (after high school and not including associate's degree)",2.422174
have not received a degree or certificate,2.448821
legitimate skip,1.953136
master's degree,3.348722
"professional doctorate (MD, JD, LLB, DDS, etc.)",3.528857


Did students who had greater academic success in hs(gpa above 3), express a desire for more education?

In [28]:
desired_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID','H4ED8'])
desired_educ = desired_educ.rename(columns = {'H4ED8': 'Value'})
expected_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID','H4ED9'])
expected_educ = expected_educ.rename(columns = {'H4ED9': 'Value_2'})
high_educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4ED2'])
high_educ = high_educ.rename(columns = {'H4ED2': 'Value_3'})

In [29]:
desired_educ_level = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'desired_educ_level')
desired_educ_level = desired_educ_level.rename(columns = desired_educ_level.iloc[0])
desired_educ_level = desired_educ_level.drop(desired_educ_level.index[0])
desired_educ_level = desired_educ_level[['Value', 'Description']]
desired_educ_level

,Value,Description
1,1,I have achieved my desired level of education
2,2,I have not achieved my desired level of educat...
3,3,I have not achieved my desired level of educat...
4,6,refused
5,8,don't know


In [30]:
expected_educ_level = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'expected_educ_level')
expected_educ_level = expected_educ_level.rename(columns = expected_educ_level.iloc[0])
expected_educ_level = expected_educ_level.drop(expected_educ_level.index[0])
expected_educ_level = expected_educ_level.rename(columns = {'Value':'Value_2'})
expected_educ_level

,Frequency,Percent,Value_2,Description
1,301,5.9%,1,finish high school or earn GED or certificate
2,179,3.5%,2,vocational/technical school after high school-...
3,189,3.7%,3,vocational/technical school after high school-...
4,286,5.6%,4,college program--less than 2 years
5,450,8.8%,5,associate's degree
6,992,19.4%,6,bachelor's degree
7,1063,20.8%,7,master's degree or equivalent
8,261,5.1%,8,"PhD or equivalent (EDD, DrPH, etc.)"
9,153,3.0%,9,"professional doctorate--MD, JD, LLB, DDS, or e..."
10,3,0.1%,96,refused


In [31]:
desired_educ_level['Value'] = desired_educ_level['Value'].astype(int)
desired_educ_level.dtypes

Value           int32
Description    object
dtype: object

In [32]:
expected_educ_level['Value_2'] = expected_educ_level['Value_2'].astype(int)
expected_educ_level.dtypes

Frequency      object
Percent        object
Value_2         int32
Description    object
dtype: object

In [33]:
desired_educ = pd.merge(desired_educ, desired_educ_level, on = 'Value', how = 'left' )
desired_educ = desired_educ.rename(columns = {'Description': 'desired_educ_level'})
desired_educ

,AID,Value,desired_educ_level
0,57101310,2,I have not achieved my desired level of educat...
1,57103869,3,I have not achieved my desired level of educat...
2,57109625,3,I have not achieved my desired level of educat...
3,57111071,1,I have achieved my desired level of education
4,57113943,2,I have not achieved my desired level of educat...
...,...,...,...
5109,99719930,2,I have not achieved my desired level of educat...
5110,99719939,2,I have not achieved my desired level of educat...
5111,99719970,2,I have not achieved my desired level of educat...
5112,99719976,1,I have achieved my desired level of education


In [34]:
expected_educ = pd.merge(expected_educ, expected_educ_level, how = 'left', on = 'Value_2')
expected_educ = expected_educ[['AID', 'Value_2', 'Description']]
expected_educ

,AID,Value_2,Description
0,57101310,7,master's degree or equivalent
1,57103869,7,master's degree or equivalent
2,57109625,1,finish high school or earn GED or certificate
3,57111071,97,legitimate skip
4,57113943,6,bachelor's degree
...,...,...,...
5109,99719930,7,master's degree or equivalent
5110,99719939,7,master's degree or equivalent
5111,99719970,6,bachelor's degree
5112,99719976,97,legitimate skip


In [35]:
expected_educ = expected_educ.rename(columns = {'Description':'expected_educ_level'})
expected_educ

,AID,Value_2,expected_educ_level
0,57101310,7,master's degree or equivalent
1,57103869,7,master's degree or equivalent
2,57109625,1,finish high school or earn GED or certificate
3,57111071,97,legitimate skip
4,57113943,6,bachelor's degree
...,...,...,...
5109,99719930,7,master's degree or equivalent
5110,99719939,7,master's degree or equivalent
5111,99719970,6,bachelor's degree
5112,99719976,97,legitimate skip


In [36]:
highest_educ = highest_educ[['Value', 'Label']]
highest_educ

,Value,Label
1,1,8th grade or less
2,2,some high school
3,3,high school graduate
4,4,some vocational/technical training (after high...
5,5,completed vocational/technical training (after...
6,6,some college
7,7,completed college (bachelor's degree)
8,8,some graduate school
9,9,completed a master's degree
10,10,some graduate training beyond a master's degree


In [37]:
highest_educ = highest_educ.rename(columns = {'Value': 'Value_3', 'Label':'highest_educ_level'})
highest_educ

,Value_3,highest_educ_level
1,1,8th grade or less
2,2,some high school
3,3,high school graduate
4,4,some vocational/technical training (after high...
5,5,completed vocational/technical training (after...
6,6,some college
7,7,completed college (bachelor's degree)
8,8,some graduate school
9,9,completed a master's degree
10,10,some graduate training beyond a master's degree


In [38]:
highest_educ['Value_3'] = highest_educ['Value_3'].astype(int)
highest_educ.dtypes

Value_3                int32
highest_educ_level    object
dtype: object

In [39]:
highest_educ = pd.merge(high_educ, highest_educ, on = 'Value_3', how = 'left')
highest_educ

,AID,Value_3,highest_educ_level
0,57101310,3,high school graduate
1,57103869,2,some high school
2,57109625,2,some high school
3,57111071,6,some college
4,57113943,6,some college
...,...,...,...
5109,99719930,7,completed college (bachelor's degree)
5110,99719939,7,completed college (bachelor's degree)
5111,99719970,3,high school graduate
5112,99719976,6,some college


In [40]:
dfs = [hs_gpa, highest_educ, desired_educ, expected_educ]

gpa_desired_educ = reduce(lambda left, right: pd.merge(left, right, on = 'AID'), dfs)
pd.set_option('display.max_colwidth', None)
#gpa_desired_educ.isna().sum()
gpa_desired_educ

,AID,cumulative_gpa,gpa_range,Value_3,highest_educ_level,Value,desired_educ_level,Value_2,expected_educ_level
0,57103869,0.767,0-1,2,some high school,3,I have not achieved my desired level of education and do not believe\nthat I will,7,master's degree or equivalent
1,57111071,3.207,3-4,6,some college,1,I have achieved my desired level of education,97,legitimate skip
2,57118381,3.156,3-4,6,some college,2,I have not achieved my desired level of education but believe that I will,6,bachelor's degree
3,57118943,2.912,2-3,12,"some post baccalaureate professional education (e.g., law school,",1,I have achieved my desired level of education,97,legitimate skip
4,57120005,3.587,3-4,7,completed college (bachelor's degree),1,I have achieved my desired level of education,97,legitimate skip
...,...,...,...,...,...,...,...,...,...
3413,99718944,3.345,3-4,8,some graduate school,2,I have not achieved my desired level of education but believe that I will,7,master's degree or equivalent
3414,99719378,3.118,3-4,6,some college,2,I have not achieved my desired level of education but believe that I will,5,associate's degree
3415,99719930,3.548,3-4,7,completed college (bachelor's degree),2,I have not achieved my desired level of education but believe that I will,7,master's degree or equivalent
3416,99719939,2.871,2-3,7,completed college (bachelor's degree),2,I have not achieved my desired level of education but believe that I will,7,master's degree or equivalent


In [41]:
gpa_desired_educ.to_csv('gpa_education.csv')

In [42]:
gpa_desired_educ.groupby('desired_educ_level')['cumulative_gpa'].min()

desired_educ_level
I have achieved my desired level of education                                        0.143
I have not achieved my desired level of education and do not believe\nthat I will    0.000
I have not achieved my desired level of education but believe that I will            0.000
don't know                                                                           1.926
Name: cumulative_gpa, dtype: float64

In [43]:
gpa_desired_educ.groupby('desired_educ_level')['cumulative_gpa'].max()

desired_educ_level
I have achieved my desired level of education                                        4.0
I have not achieved my desired level of education and do not believe\nthat I will    4.0
I have not achieved my desired level of education but believe that I will            4.0
don't know                                                                           3.5
Name: cumulative_gpa, dtype: float64

In [44]:
gpa_desired_educ.groupby('desired_educ_level')['cumulative_gpa'].mean()

desired_educ_level
I have achieved my desired level of education                                        2.889838
I have not achieved my desired level of education and do not believe\nthat I will    2.392034
I have not achieved my desired level of education but believe that I will            2.550425
don't know                                                                           2.679500
Name: cumulative_gpa, dtype: float64

Education level for people who haven’t reached the desired level yet? Those who have?

In [45]:
desired_achieved = gpa_desired_educ.loc[gpa_desired_educ['Value'] == 1]
desired_achieved.head()

,AID,cumulative_gpa,gpa_range,Value_3,highest_educ_level,Value,desired_educ_level,Value_2,expected_educ_level
1,57111071,3.207,3-4,6,some college,1,I have achieved my desired level of education,97,legitimate skip
3,57118943,2.912,2-3,12,"some post baccalaureate professional education (e.g., law school,",1,I have achieved my desired level of education,97,legitimate skip
4,57120005,3.587,3-4,7,completed college (bachelor's degree),1,I have achieved my desired level of education,97,legitimate skip
6,57121404,3.576,3-4,9,completed a master's degree,1,I have achieved my desired level of education,97,legitimate skip
10,57131909,1.433,1-2,5,completed vocational/technical training (after high school),1,I have achieved my desired level of education,97,legitimate skip


In [46]:
desired_achieved['gpa_range'].value_counts(normalize = True)

3-4    0.514552
2-3    0.335274
1-2    0.135041
0-1    0.015134
Name: gpa_range, dtype: float64

In [47]:
desired_achieved['highest_educ_level'].value_counts(normalize = True)

completed college (bachelor's degree)                                                         0.327125
high school graduate                                                                          0.166473
some college                                                                                  0.160652
completed a master's degree                                                                   0.129220
completed vocational/technical training (after high school)                                   0.073341
completed post baccalaureate professional education (e.g., law\nschool, med school, nurse)    0.043073
some vocational/technical training (after high school)                                        0.025611
completed a doctoral degree                                                                   0.020955
some high school                                                                              0.019790
some graduate school                                                     

In [48]:
desired_educ_never = gpa_desired_educ.loc[gpa_desired_educ['Value'] == 3]
desired_educ_never.head()

,AID,cumulative_gpa,gpa_range,Value_3,highest_educ_level,Value,desired_educ_level,Value_2,expected_educ_level
0,57103869,0.767,0-1,2,some high school,3,I have not achieved my desired level of education and do not believe\nthat I will,7,master's degree or equivalent
19,57154779,0.000,0-1,2,some high school,3,I have not achieved my desired level of education and do not believe\nthat I will,1,finish high school or earn GED or certificate
21,57160648,1.340,1-2,4,some vocational/technical training (after high school),3,I have not achieved my desired level of education and do not believe\nthat I will,2,vocational/technical school after high school--less than 2 years
22,57165081,2.375,2-3,3,high school graduate,3,I have not achieved my desired level of education and do not believe\nthat I will,1,finish high school or earn GED or certificate
27,57178982,1.450,1-2,3,high school graduate,3,I have not achieved my desired level of education and do not believe\nthat I will,4,college program--less than 2 years


In [49]:
desired_educ_never['gpa_range'].value_counts(normalize = True)

2-3    0.513793
1-2    0.213793
3-4    0.206897
0-1    0.065517
Name: gpa_range, dtype: float64

In [50]:
desired_educ_never['highest_educ_level'].value_counts(normalize = True)

high school graduate                                                 0.324138
some college                                                         0.300000
some high school                                                     0.117241
completed college (bachelor's degree)                                0.106897
completed vocational/technical training (after high school)          0.072414
some vocational/technical training (after high school)               0.051724
completed a master's degree                                          0.010345
some graduate school                                                 0.010345
some post baccalaureate professional education (e.g., law school,    0.003448
completed a doctoral degree                                          0.003448
Name: highest_educ_level, dtype: float64

In [51]:
desired_will_achieve = gpa_desired_educ.loc[gpa_desired_educ['Value'] == 2]
desired_will_achieve.head()

,AID,cumulative_gpa,gpa_range,Value_3,highest_educ_level,Value,desired_educ_level,Value_2,expected_educ_level
2,57118381,3.156,3-4,6,some college,2,I have not achieved my desired level of education but believe that I will,6,bachelor's degree
5,57120371,2.386,2-3,3,high school graduate,2,I have not achieved my desired level of education but believe that I will,4,college program--less than 2 years
7,57121476,1.875,1-2,3,high school graduate,2,I have not achieved my desired level of education but believe that I will,6,bachelor's degree
8,57127241,3.042,3-4,6,some college,2,I have not achieved my desired level of education but believe that I will,5,associate's degree
9,57129567,0.843,0-1,4,some vocational/technical training (after high school),2,I have not achieved my desired level of education but believe that I will,3,vocational/technical school after high school--2 or more years


In [52]:
desired_will_achieve['gpa_range'].value_counts(normalize = True)

2-3    0.431728
3-4    0.318162
1-2    0.204596
0-1    0.045515
Name: gpa_range, dtype: float64

In [53]:
desired_will_achieve['highest_educ_level'].value_counts(normalize = True)

some college                                                                                  0.407424
completed college (bachelor's degree)                                                         0.190013
high school graduate                                                                          0.122846
completed vocational/technical training (after high school)                                   0.068051
some graduate school                                                                          0.056120
some high school                                                                              0.050376
some vocational/technical training (after high school)                                        0.035793
completed a master's degree                                                                   0.035351
some graduate training beyond a master's degree                                               0.016792
some post baccalaureate professional education (e.g., law school,        

opinions on educational attainment

In [54]:
#educ_views = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0005\21600-0005-Data.tsv", sep = '\t', usecols = ['AID','H2EE1', 'H2EE2', 'H2EE16'])
#educ_views

In [55]:
#want_college = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 2 Questionaire.xlsx", 'H2EE1')
#college_likely = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 2 Questionaire.xlsx", 'H2EE2')
#graduation_likely = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 2 Questionaire.xlsx", 'H2EE16')

In [56]:
#educ_views = pd.merge(educ_views, want_college, how = 'left', on = 'H2EE1')
#educ_views = pd.merge(educ_views, college_likely, how = 'left', on = 'H2EE2')
#educ_views = pd.merge(educ_views, graduation_likely, how = 'left', on = 'H2EE16')
#educ_views = pd.merge(AID, educ_views, how = 'left', on = 'AID')
#educ_views

In [57]:
#educ_views_level = pd.merge(educ_views, highest_educ, on = 'AID', how = 'left')

In [58]:
#educ_views_level.to_csv('educ_views_level.csv')

demographics -- race

In [59]:
race = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID', 'H1GI6A', 'H1GI6B', 'H1GI6C', 'H1GI6D', 'H1GI6E'])
race = race.rename(columns = {'H1GI6A': 'White', 'H1GI6B': 'Black or African American', 'H1GI6C': 'American Indian or Native American', 'H1GI6D': 'Asian or Pacific Islander', 'H1GI6E': 'Other'})
race = pd.merge(AID, race, how = 'left', on = 'AID')
race

,AID,White,Black or African American,American Indian or Native American,Asian or Pacific Islander,Other
0,57103869,0,1,0,0,0
1,57111071,1,0,0,0,0
2,57117542,1,0,0,0,0
3,57118381,1,0,0,0,0
4,57118943,1,0,0,0,0
...,...,...,...,...,...,...
3437,99718944,1,0,0,0,0
3438,99719378,1,0,0,0,0
3439,99719930,0,1,0,0,0
3440,99719939,0,1,0,0,0


In [60]:
race.to_csv("race.csv")

In [61]:
#Checking to see if people who refused to answer or said don't know, associated themselves with any race
#people who refused to answer, will remove them from the dataset and they're the same people
#people who don't know, will remove them from the dataset as well

#race.loc[race['White'] == 8]
#race.loc[race['Black or African American'] == 8]
#race.loc[race['American Indian or Native American'] == 8]
#race.loc[race['Asian or Pacific Islander'] == 8]
race = race.drop(race[(race['Other'] == 8) | (race['Other'] == 6)].index)
race.head()

,AID,White,Black or African American,American Indian or Native American,Asian or Pacific Islander,Other
0,57103869,0,1,0,0,0
1,57111071,1,0,0,0,0
2,57117542,1,0,0,0,0
3,57118381,1,0,0,0,0
4,57118943,1,0,0,0,0


In [62]:
#add totals row to get the sum of each row
race.loc['Total'] = race.iloc[:, 1:].sum() 
race.loc[:, 'Col_Total'] = race.iloc[:, 1:].sum(axis=1)

In [63]:
race

,AID,White,Black or African American,American Indian or Native American,Asian or Pacific Islander,Other,Col_Total
0,57103869.0,0.0,1.0,0.0,0.0,0.0,1.0
1,57111071.0,1.0,0.0,0.0,0.0,0.0,1.0
2,57117542.0,1.0,0.0,0.0,0.0,0.0,1.0
3,57118381.0,1.0,0.0,0.0,0.0,0.0,1.0
4,57118943.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
3438,99719378.0,1.0,0.0,0.0,0.0,0.0,1.0
3439,99719930.0,0.0,1.0,0.0,0.0,0.0,1.0
3440,99719939.0,0.0,1.0,0.0,0.0,0.0,1.0
3441,99719976.0,1.0,0.0,0.0,0.0,0.0,1.0


In [64]:
race.to_csv("race.csv")

In [65]:
#demographics: merge with key to know what the value stands for
#merge all demographics into one table

demographics -- household income

In [66]:
income = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID','H4EC1', 'H4EC2'])
income = income.rename(columns = {'H4EC1': 'household_income', 'H4EC2': 'personal_income'})
income = pd.merge(AID, income, how = 'left', on = 'AID')
income

,AID,household_income,personal_income
0,57103869,1,9999998
1,57111071,6,27000
2,57117542,9,5000
3,57118381,9,19000
4,57118943,11,40000
...,...,...,...
3437,99718944,10,44000
3438,99719378,8,42000
3439,99719930,7,36000
3440,99719939,4,15000


In [67]:
income['personal_income'].describe()

count    3.442000e+03
mean     4.614551e+05
std      2.015453e+06
min      0.000000e+00
25%      1.700000e+04
50%      3.100000e+04
75%      4.800000e+04
max      9.999998e+06
Name: personal_income, dtype: float64

demographics -- debt

In [68]:
debt = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4EC8'])
debt = debt.rename(columns = {'H4EC8': 'household_debt'})
debt = pd.merge(AID, debt, how = 'left', on = 'AID')
debt

,AID,household_debt
0,57103869,1
1,57111071,5
2,57117542,4
3,57118381,4
4,57118943,2
...,...,...
3437,99718944,7
3438,99719378,5
3439,99719930,2
3440,99719939,2


demographics -- children

In [69]:
children = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4TR11'])
children = children.rename(columns = {'H4TR11': 'children'})
children = pd.merge(AID, children, how = 'left', on = 'AID')
children

,AID,children
0,57103869,97
1,57111071,97
2,57117542,1
3,57118381,3
4,57118943,97
...,...,...
3437,99718944,97
3438,99719378,97
3439,99719930,97
3440,99719939,97


demographics -- married

In [70]:
married = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4TR1'])
married = married.rename(columns = {'H4TR1': 'times_married'})
married = pd.merge(AID, married, how = 'left', on = 'AID')
married

,AID,times_married
0,57103869,0
1,57111071,0
2,57117542,0
3,57118381,1
4,57118943,1
...,...,...
3437,99718944,0
3438,99719378,1
3439,99719930,0
3440,99719939,0
